In [ ]:
import sys
import tensorflow as tf
from tensorflow.keras import layers, activations, losses, Model, Input
from tensorflow.nn import leaky_relu
import numpy as np
from itertools import combinations
from tensorflow.keras.utils import plot_model, Progbar
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [ ]:

# In case your sys.path does not contain the base repo, cd there.
print(sys.path)
%cd '~/ml-solr-course'

The idea behind RankNet is to model the **joint probability** that `document i` comes before `document j` as the following:

$P_{ij} = 1$ if $s_i > s_j$
$P_{ij} = 0.5$ if $s_i = s_j$
$P_{ij} = 0$ if $s_i < s_j$

So for *every pair of inputs* we will calculate both outputs, substract them, pass a logistic function to model the probability:

<img src="files/ranknet.png">

In [ ]:
# Try to create a model with 2 dense layers with leaky_relu as activations. Then a linear dense function and a substract layer.

# This time I will leave it blank, but in the parameter oij you should have the output of the substraction.

# model architecture
class RankNet(Model):
    def __init__(self):
        super().__init__()
        # FILL

    def call(self, inputs):
        xi, xj = inputs
        # FILL
        output = layers.Activation('sigmoid')(oij)
        return output

    def build_graph(self):
        x = [Input(shape=(10)), Input(shape=(10))]
        return Model(inputs=x, outputs=self.call(x))

In [ ]:
#Generate random data
nb_query = 20
query = np.array([i+1 for i in range(nb_query) for x in range(int(np.ceil(np.abs(np.random.normal(0,scale=15))+2)))])
doc_features = np.random.random((len(query), 10))
doc_scores = np.random.randint(5, size=len(query)).astype(np.float32)



In [ ]:
# put data into pairs
xi = []
xj = []
pij = []
pair_id = []
pair_query_id = []
for q in np.unique(query):
    query_idx = np.where(query == q)[0]
    for pair_idx in combinations(query_idx, 2):
        pair_query_id.append(q)

        pair_id.append(pair_idx)
        i = pair_idx[0]
        j = pair_idx[1]
        xi.append(doc_features[i])
        xj.append(doc_features[j])

        if doc_scores[i] == doc_scores[j]:
            _pij = 0.5
        elif doc_scores[i] > doc_scores[j]:
            _pij = 1
        else:
            _pij = 0
        pij.append(_pij)

xi = np.array(xi)
xj = np.array(xj)
pij = np.array(pij)
pair_query_id = np.array(pair_query_id)

In [ ]:
# Use the train test split method from sklearn to split 20% into a validation set to feed keras.
xi_train, xi_test, xj_train, xj_test, pij_train, pij_test, pair_id_train, pair_id_test = train_test_split(
    xi, xj, pij, pair_id, test_size=0.2, stratify=pair_query_id)

In [ ]:
# train model using compile with binary_crossentropy and fit. Pass as inputs [xi_train, xj_train] and pij_train.
ranknet = RankNet()
history = None # Fill

In [ ]:
# function for plotting loss
def plot_metrics(train_metric, val_metric=None, metric_name=None, title=None, ylim=5):
    plt.title(title)
    plt.ylim(0,ylim)
    plt.plot(train_metric,color='blue',label=metric_name)
    if val_metric is not None: plt.plot(val_metric,color='green',label='val_' + metric_name)
    plt.legend(loc="upper right")

# plot loss history
plot_metrics(history.history['loss'], history.history['val_loss'], "Loss", "Loss", ylim=1.0)

In [ ]:
#Test with a a new sample pair of docs to get their associated probability.

new_doci = None
new_docj = None
inputs = None
ranknet(inputs)